In [ ]:
# ✅ 머신러닝 뜻 (마케터 버전)
# 데이터를 기반으로 컴퓨터가 패턴을 스스로 학습해서 예측하거나 자동으로 최적화하는 기술

# 쉽게 말하면:
# 👉 사람이 규칙을 일일이 정하는 게 아니라
# 👉 데이터 보고 “알아서” 예측하고 최적화하는 것

# 머신러닝을위한통계와확률


# ch4. 선형회귀 모델
# 변수 이해
# 관심변수와 설명변수(독립변수)
# 관심변수: 예측하고자하는대상: 종속변수,타깃변수,출력변수
# 설명변수:관심변수에 영향을 미치는 요인= 독립변수,피쳐,입력변수 


In [ ]:
# pip install scikit-learn  아나콘다 설치

In [6]:
import pandas as pd
import numpy as np

# 그래프 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns

# 회귀 라이브러리
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm    # 통계적 해석 
from sklearn.metrics import r2_score

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

# 1. 선형회귀 모델의 이해

In [2]:
# 보험료(charges) 예측 선형회귀 모델 만들기

In [11]:
df_ins = pd.read_csv('data/insurance.csv')
df_ins.shape
df_ins.head(3)

,age,sex,bmi,children,smoker,region,charges,ins_type,period
0,18,male,53.13,0,no,southeast,1163.4627,A,7.4
1,22,male,52.58,1,yes,southeast,44501.3982,B,6.5
2,23,male,50.38,1,no,southeast,2438.0552,C,4.5


In [8]:
# X : 설명변수/독립변수 ,   y: 관심변수/종속변수
X = df_ins[ ['age', 'bmi', 'children', 'period'] ]
y = df_ins['charges'] #찰지스 (비용) 

In [14]:
#LinearRegression =선형회귀(리니어 리그레션)

# 💡 마케팅 실무 비유
# 광고비(x) → 매출(y) 예측
# 절편(intercept) = 광고비 0일 때 기본 매출  즉, 광고를 안 해도 발생하는 매출 수준
#🎯 핵심 한 줄절편(intercept) = x=0일 때 y값, 직선이 y축을 만나는 지점
model = LinearRegression(fit_intercept=True)  # 상수항 추가(모델을 유연하게 만들어줌)
model

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [15]:
# 선형회귀 모델 적합(fit)
# model → 아까 만든 LinearRegression 모델
# fit() → “모델을 데이터에 맞춘다 / 학습한다”
# X → 독립변수 / 설명 변수
# y → 종속변수 / 목표값

model.fit(X, y)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [16]:
print(model.intercept_)  # 절편(상수항)   intercept = 절편 모든 것이 0일 때 시작점
# 예를 들어:
# 네가 자전거 타기 게임을 하는데,
# 처음 위치(시작점)가 2칸 앞이면,
# intercept = 2

즉, x가 0일 때 y가 어디서 시작하는지 알려주는 숫자야.
print(model.coef_)       # 회귀계수  쉽게 말하면: 하나를 더하면 y가 얼마나 움직이는지 알려주는 숫자

-7959.161078311678
[233.47906496 337.36480124 555.55931003 209.69390419]


In [17]:
# y = 233.4*x1 + 337.3*x2 + 555.55*x3 + 209.69*x4 - 7959

# 보험료 예측  / predict = 학습한 모델로 새로운 데이터의 결과(y) 예측하기
model.predict([[25, 33, 2, 4.5]]) #[나이 근무시간 부서코드 평점)

array([11065.59517576])

In [18]:
# R2 score(설명력) : 값이 1에 가까울수록 모델이 데이터를 잘 설명한다고 볼 수 있음
# r2_score = 결정계수(R²)
# 의미: 모델이 실제 y값을 얼마나 잘 맞췄는지 알려주는 숫자
# 0 ~ 1 사이 값 (1에 가까울수록 완벽하게 예측)

from sklearn.metrics import r2_score

y_pred = model.predict(X)  # 모델 예측값 / 학습한 모델로 X에 대한 예측값(y_pred) 계산
r2_score(y, y_pred) #→ 실제값(y)과 예측값(y_pred)을 비교해서 얼마나 잘 맞았는지 점수
#r2_score  보통0~1  지금 결과값 0.12 너무 낮다 왜그럴까>? 범주형 변수를 넣지 않아서 범수형 변수를 수치화해서 다시해야함

0.12025303515697039

# 2. 변수 선택과 모델 성능

### 2-1. 범주형 변수의 수치화

In [10]:
# 범주형 변수(sex, smoker, region, ins_type)들도 회귀 모델에 추가하기

In [19]:
df_ins.head(3)

,age,sex,bmi,children,smoker,region,charges,ins_type,period
0,18,male,53.13,0,no,southeast,1163.4627,A,7.4
1,22,male,52.58,1,yes,southeast,44501.3982,B,6.5
2,23,male,50.38,1,no,southeast,2438.0552,C,4.5


In [20]:
# pd.get_dummies() 함수 활용 : 범주형 변수들 -> 수치형 변수  원핫인코딩(one-hot encoding) : 범주형 변수를 0 또는 1로 표현

# pd.get_dummies() → 범주형(object) 변수를 0,1 숫자로 변환
# data=df_ins → 변환할 원본 데이터프레임
# dtype='int' → 숫자형 자료로 변환
# drop_first=True → 기준 범주 하나는 빼서 중복 방지

df_dummies = pd.get_dummies(data=df_ins, dtype='int', drop_first=True)   
df_dummies.head(5)

,age,bmi,children,charges,period,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest,ins_type_B,ins_type_C
0,18,53.13,0,1163.4627,7.4,1,0,0,1,0,0,0
1,22,52.58,1,44501.3982,6.5,1,1,0,1,0,1,0
2,23,50.38,1,2438.0552,4.5,1,0,0,1,0,0,1
3,58,49.06,0,11381.3254,6.6,1,0,0,1,0,1,0
4,46,48.07,2,9432.9253,4.0,0,0,0,0,0,0,1


In [28]:
# X : 설명변수/독립변수    # y: 관심변수/종속변수 
X = df_dummies.drop(columns=['charges']) #df_dummies에서'charges' 컬럼만 제외하고 나머지 전부 X에 담음의미: 모델이 학습할 입력 데이터
y = df_dummies['charges']    #df_dummies에서 'charges' 컬럼만 y에 담음 의미: 예측하려는 값

In [22]:
# 선형회귀 모델 적합(fit)
linear_model = LinearRegression(fit_intercept=True)
linear_model.fit(X, y)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [23]:
print(linear_model.intercept_)  #상수항
print(linear_model.coef_)  # 회귀계수

-12068.22947330276
[  250.88802329   345.84717412   463.55011366    71.14446418
  -210.97951778 22730.98447422  -396.30451611 -1750.58599023
 -1008.13753106    77.00625202   228.35874577]


In [24]:
# R2 score(설명력) : 값이 1에 가까울수록 모델이 데이터를 잘 설명한다고 볼 수 있음
from sklearn.metrics import r2_score

y_pred = linear_model.predict(X)  # 모델 예측값
r2_score(y, y_pred)

0.7164991724033529

### 2-2. 선형회귀 모델의 R2 score와 회귀계수

In [27]:
import statsmodels.api as sm

X_sm = sm.add_constant(X)   # 상수항 추가(모델을 유연하게 만들어줌)
X_sm

,const,age,bmi,children,period,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest,ins_type_B,ins_type_C
0,1.0,18,53.130,0,7.4,1,0,0,1,0,0,0
1,1.0,22,52.580,1,6.5,1,1,0,1,0,1,0
2,1.0,23,50.380,1,4.5,1,0,0,1,0,0,1
3,1.0,58,49.060,0,6.6,1,0,0,1,0,1,0
4,1.0,46,48.070,2,4.0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1348,1.0,28,17.290,0,3.4,0,0,0,0,0,0,1
1349,1.0,26,17.195,2,3.5,0,1,0,0,0,0,1
1350,1.0,21,16.815,1,2.2,0,0,0,0,0,0,1
1351,1.0,38,16.815,2,4.1,1,0,0,0,0,1,0


In [26]:
# 선형회귀 모델 적합(fit)
ls = sm.OLS(y, X_sm).fit()  #sm.OLS → 선형회귀 모델 만들기 / .fit() → 모델 학습
ls.summary() # → 학습 결과 표로 보기

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                charges   R-squared:                       0.716
Model:                            OLS   Adj. R-squared:                  0.714
Method:                 Least Squares   F-statistic:                     308.1
Date:                Wed, 11 Feb 2026   Prob (F-statistic):               0.00
Time:                        15:44:05   Log-Likelihood:                -13772.
No. Observations:                1353   AIC:                         2.757e+04
Df Residuals:                    1341   BIC:                         2.763e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -1.207e+04   1169.484    -10.319      0.000   -1.44e+04   -9774.013
age                250.8880     12.549     19.992      0.000     226.270     275.506
bmi                345.8472     30.745     11.249      0.000     285.534     406.160
children           463.5501    145.420      3.188      0.001     178.275     748.826
period              71.1445     87.343      0.815      0.415    -100.200     242.489
sex_male          -210.9795    350.370     -0.602      0.547    -898.312     476.353
smoker_yes        2.273e+04    430.472     52.805      0.000    2.19e+04    2.36e+04
region_northwest  -396.3045    507.073     -0.782      0.435   -1391.047     598.438
region_southeast -1750.5860    506.397     -3.457      0.001   -2744.002    -757.170
region_southwest -1008.1375    505.478     -1.994      0.046   -1999.751     -16.524
ins_type_B          77.0063    431.504      0.178      0.858    -769.491     923.503
ins_type_C         228.3587    429.395      0.532      0.595    -614.001    1070.718
==============================================================================
Omnibus:                      217.162   Durbin-Watson:                   2.152
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              537.770
Skew:                           0.873   Prob(JB):                    1.68e-117
Kurtosis:                       5.548   Cond. No.                         352.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
#위 그래프 피벨류 높으면 결고값에 영향 /낮으면 x